# Simulateur d'espace

Ce workflow a pour but de simuler l'espace disponible sur chacun des racks afin de permettre une projection des équipes dans le déplacement du matériel.

SELECT it.nom, it.short_name, it.long_name, it.ASSET_ID, it.Nom_de_la_salle , it.Nom_de_baie, it.Slot_U, it.type
from `IT_Equipment_v2` it

`rack_capacity_map` : cartographie les racks de toutes les salles afin de rajouter la capacité maximale d'un rack défini à -1 unités U. Si -1 le rack n' est pas dispo sur le fichier initial cabinet

In [ ]:
drop table if exists rack_capacity_map ;
create table rack_capacity_map
as 
SELECT it.Nom_de_la_salle, it.Nom_de_baie, '-1' as rack_capacity
from `IT_Equipment_v2` it
GROUP BY it.Nom_de_la_salle, it.Nom_de_baie ;

In [ ]:
INSERT INTO `rack_capacity_map`
SELECT it2.Nom_de_la_salle, it2.nom, '-1' as rack_capacity from `rack_capacity_map` it 
RIGHT JOIN `IT_Equipment_rack_description` it2 ON concat(it2.Nom_de_la_salle, it2.nom) = concat(it.Nom_de_la_salle, it.nom_de_baie)
where it.nom_de_baie is null

In [ ]:
update `rack_capacity_map` it
inner join `IT_Equipment_rack_description` it2 ON it.Nom_de_la_salle = it2.Nom_de_la_salle and it.nom_de_baie = it2.nom
set it.rack_capacity = it2.Nombre_de_U

`indicator_it_equipment_simulateur_espace` : indicateur de l'espace disponible pour tous lers rack par rapport à la somme des hauteurs de chaque equipment renseigné sur la CMDB

drop table if exists indicator_it_equipment_simulateur_espace_raw ;
create table indicator_it_equipment_simulateur_espace_raw
as
SELECT concat(it.Nom_de_la_salle, it.Nom_de_baie) key_id, it.Nom_de_la_salle, it.Nom_de_baie, r.rack_capacity, sum(CEILING(it.Hauteur_U)) slot_occupied, r.rack_capacity - sum(CEILING(it.Hauteur_U)) free_slot,
GROUP_CONCAT(concat(it.slot_u, '/', CEILING(it.Hauteur_U))) list_occupied_slot
from `IT_Equipment_v2` it
INNER JOIN `rack_capacity_map` r ON r.Nom_de_la_salle = it.Nom_de_la_salle and r.Nom_de_baie = it.Nom_de_baie
GROUP BY it.Nom_de_la_salle, it.Nom_de_baie
ORDER by it.Nom_de_la_salle desc , sum(CEILING(it.Hauteur_U)) desc
;

In [ ]:
drop table if exists indicator_it_equipment_simulateur_espace_raw ;
create table indicator_it_equipment_simulateur_espace_raw
as
SELECT if(it.Nom_de_baie is not null, concat(it.Nom_de_la_salle, it.Nom_de_baie), concat(r.Nom_de_la_salle, r.Nom_de_baie))  key_id,
if(it.Nom_de_la_salle is not null, it.Nom_de_la_salle, r.Nom_de_la_salle) Nom_de_la_salle,
if(it.Nom_de_baie is not null, it.Nom_de_baie, r.Nom_de_baie) Nom_de_baie,
r.rack_capacity, 
if(sum(CEILING(it.Hauteur_U)) is not null, sum(CEILING(it.Hauteur_U)), 0) slot_occupied,
if(sum(CEILING(it.Hauteur_U)) is not null, r.rack_capacity - sum(CEILING(it.Hauteur_U)), r.rack_capacity) free_slot,
if(sum(CEILING(it.Hauteur_U)) is not null, GROUP_CONCAT(concat(it.slot_u, '/', CEILING(it.Hauteur_U))), '') list_occupied_slot
from `IT_Equipment_v2` it
RIGHT JOIN `rack_capacity_map` r ON r.Nom_de_la_salle = it.Nom_de_la_salle and r.Nom_de_baie = it.Nom_de_baie

GROUP BY r.Nom_de_la_salle, r.Nom_de_baie
-- ORDER by it.Nom_de_la_salle desc , sum(CEILING(it.Hauteur_U)) desc

alter table indicator_it_equipment_simulateur_espace_raw
ADD COLUMN occupied_item varchar(255),
ADD COLUMN free_item varchar(255);

drop view if exists indicator_it_equipment_simulateur_espace ;
create view indicator_it_equipment_simulateur_espace
as
SELECT concat(it.Nom_de_la_salle, it.Nom_de_baie) key_id, it.Nom_de_la_salle, it.Nom_de_baie, r.rack_capacity, sum(CEILING(it.Hauteur_U)) slot_occupied, r.rack_capacity - sum(CEILING(it.Hauteur_U)) free_slot
from `IT_Equipment_v2` it
INNER JOIN `rack_capacity_map` r ON r.Nom_de_la_salle = it.Nom_de_la_salle and r.Nom_de_baie = it.Nom_de_baie
GROUP BY it.Nom_de_la_salle, it.Nom_de_baie
ORDER by it.Nom_de_la_salle desc , sum(CEILING(it.Hauteur_U)) desc
;

class indicator_simulateur_espace(models.Model):   
    id = models.BigAutoField(primary_key=True)
    Nom_de_la_salle = models.AutoField()  # Field name made lowercase.
    Nom_de_baie = models.CharField(max_length=255)
    rack_capacity = models.CharField(max_length=255)
    slot_occupied = models.CharField(max_length=255)
    free_slot = models.CharField(max_length=255)
    class Meta:
        managed = False
        db_table = "indicator_it_equipment_simulateur_espace"

SELECT it.nom, it.short_name, it.long_name, it.ASSET_ID, it.Nom_de_la_salle , it.Nom_de_baie, it.Slot_U, it.type, it.Hauteur_U, 
it.Slot_U + it.Hauteur_U tt,
concat()

from `IT_Equipment_v2` it
-- where Nom_de_la_salle = 'P4' and Nom_de_baie = 'AE17'
where Nom_de_la_salle = 'P1' and Nom_de_baie = 'A26'

CREATE TABLE `indicator_it_equipment_simulateur_espace` (
  `key_id` varchar(255) ,
  `Nom_de_la_salle` varchar(255) DEFAULT NULL,
  `Nom_de_baie` varchar(255) DEFAULT NULL,
  `rack_capacity` varchar(100) NOT NULL,
  `slot_occupied` varchar(100) NOT NULL,
  `free_slot`varchar(255) NOT NULL,
  `occupied_item` varchar(100) NOT NULL,
  `free_item` varchar(100) NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8;